In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [66]:
import numpy as np
import pandas as pd
import cv2
import os
import torch
import fastai
from fastai import *
from fastai.vision import *
from fastai.vision.all import *
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

In [64]:
!pip install fastai --upgrade

  Using cached fastai-2.2.7-py3-none-any.whl (193 kB)
  Using cached torch-1.7.1-cp37-cp37m-win_amd64.whl (184.1 MB)
  Using cached fastai-2.2.6-py3-none-any.whl (193 kB)
  Using cached fastai-2.2.5-py3-none-any.whl (191 kB)
  Using cached fastai-2.2.4-py3-none-any.whl (191 kB)
  Using cached fastai-2.2.3-py3-none-any.whl (191 kB)
  Using cached fastai-2.2.2-py3-none-any.whl (191 kB)
  Using cached fastai-2.2.1-py3-none-any.whl (191 kB)
  Using cached fastai-2.2.0-py3-none-any.whl (191 kB)


In [32]:
path = Path('./data/faces')
output_path = './data/testing'

In [33]:
df = pd.read_csv(path/'../../datasets/labels.csv')

In [34]:
dls = ImageDataLoaders.from_df(df, path, label_delim=' ',
                               item_tfms=Resize(460), batch_tfms=aug_transforms(size=224),num_workers=0)

In [38]:
# Loading our model
learn = cnn_learner(dls, models.resnet50, pretrained=False)
learn.load("ff_stage-2-rn50")

In [42]:
 face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [100]:
import glob
import imutils
from tqdm.notebook import tqdm
from fastai.vision import *
itemlist = glob.glob('./data/testing/*.jpg')
fc = face_cascade
outputs = []
wrinkles = ["no_wrinkles","wrinkles"]
freakles = ["no_freakles","freakles"]
glasses = ["no_glasses","normal_glasses","sunglasses"]
hair_color = ["brown","black","gray","blond","red","white","mixed","other"]
top = ["bald_or_shaved","has_few_hairs","has_thick_hair"," "]
for item in tqdm(itemlist):
     ## Importing image using open cv
    img = cv2.imread(item)
    ## Resizing to constant width
    img = imutils.resize(img, width=200)
    
    ## Finding actual size of image
    H,W,_ = img.shape
    
    ## Converting BGR to RGB
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ## Detecting faces on the image
    face_coord = fc.detectMultiScale(gray,1.2,10,minSize=(50,50))
    
    ## If only one face is found
    if len(face_coord) == 1:
        X, Y, w, h = face_coord[0]
    
    ## If no face found --> SKIP
    elif len(face_coord)==0:
        continue
    
    ## If multiple faces are found take the one with largest area
    else:
        max_val = 0
        max_idx = 0
        for idx in range(len(face_coord)):
            _, _, w_i, h_i = face_coord[idx]
            if w_i*h_i > max_val:
                max_idx = idx
                max_val = w_i*h_i
            else:
                pass
            
            X, Y, w, h = face_coord[max_idx]
    
    ## Crop and export the image
    img_cp = img[
            max(0,Y - int(0.35*h)): min(Y + int(1.35*h), H),
            max(0,X - int(w*0.35)): min(X + int(1.35*w), W)
        ].copy()
    ## Prediction of facial featues
    temp = []
    img_fastai = torch.from_numpy(np.array(img_cp,copy = False))
    prediction = str(
        learn.predict(img_fastai)[0]
    ).split(";")[0]
    cnt = 0
    #print(item)
    temp.append(item.split('\\')[1])
    for item in wrinkles:
        if item in prediction:
            temp.append(cnt)
            break
        cnt+=1
    cnt = 0
    for item in freakles:
        if item in prediction:
            temp.append(cnt)
            break
        cnt+=1
    cnt = 0
    for item in glasses:
        if item in prediction:
            temp.append(cnt)
            break
        cnt+=1
    cnt = 0
    for item in hair_color:
        if item in prediction:
            temp.append(cnt)
            break
        cnt+=1
    cnt = 0
    for item in top:
        if item in prediction:
            temp.append(cnt)
            break
        cnt+=1
    outputs.append(temp)
    print(prediction)
headers = ["image_name","wrinkles","freakles","glasses","hair_color","hair_top"]
pd.DataFrame(outputs).to_csv("output.csv",header = headers , index = None)

  0%|          | 0/5 [00:00<?, ?it/s]

['black', 'has_thick_hair', 'no_freakles', 'no_glasses', 'no_wrinkles']


['black', 'has_thick_hair', 'no_freakles', 'no_glasses', 'no_wrinkles']


['black', 'has_thick_hair', 'no_freakles', 'no_glasses', 'no_wrinkles']


['has_few_hairs', 'no_freakles', 'no_glasses', 'no_wrinkles', 'white']


['black', 'has_thick_hair', 'no_freakles', 'no_glasses', 'no_wrinkles']
